# 야식(배달) 데이터 전처리
## 평균 주문 금액

In [1]:
import pandas as pd

In [3]:
# 데이터 불러오기
column_names = ['날짜','시간대별 시간','배달상점 업종명','목적지 광역시도명','목적지 시군구명','평균주문금액']
df = pd.read_csv('data/raw/KGU_3rd_ORIGIN_KGUDSTNAMOUNT_20210101000000.csv', names=column_names, header=None)
df.head()

,날짜,시간대별 시간,배달상점 업종명,목적지 광역시도명,목적지 시군구명,평균주문금액
0,2021-01-01,0,돈까스/일식,경기도,동두천시,72667
1,2021-01-01,0,돈까스/일식,경기도,부천시,34877
2,2021-01-01,0,돈까스/일식,경기도,안산시 상록구,24420
3,2021-01-01,0,돈까스/일식,경기도,의정부시,31600
4,2021-01-01,0,돈까스/일식,경상남도,진주시,45333


In [4]:
# 서울, 경기, 인천 필터링
order_price_df = df[(df['목적지 광역시도명'].isin(['서울특별시', '경기도', '인천광역시']))]

2021년도 날짜 맞추기

In [5]:
order_price_df['날짜'].dtype

dtype('O')

In [ ]:
order_price_df['날짜'] = order_price_df['날짜'].str.split('-').str[0]

야간 배달 필터링
- 기준 시간 (20~06시)

In [9]:
order_price_df = order_price_df[order_price_df['시간대별 시간'].isin(list(range(20, 24)) + list(range(0, 5)))]

In [10]:
# 필요없는 컬럼 삭제
order_price_df.drop(['배달상점 업종명'], axis=1, inplace=True)

시군구2 : 경기/인천 '구' 단위 → '시' 단위 통합

In [12]:
def replace_values_with_multiple_substrings(order_price_df, column, replacements):
  for old_value, new_value in replacements.items():
        order_price_df.loc[order_price_df[column].str.contains(old_value, na=False), column] = new_value
  return order_price_df

replacements = {
    '수원시': '수원시',
    '성남시': '성남시',
    '안양시': '안양시',
    '부천시': '부천시',
    '안산시': '안산시',
    '고양시': '고양시',
    '용인시': '용인시',
}
order_price_df['시군구2'] = order_price_df['목적지 시군구명']
order_price_df = replace_values_with_multiple_substrings(order_price_df, '시군구2', replacements)

In [21]:
order_price_df.to_csv('data1/order_price.csv')